# Module 04 - Tidy data 
 What is Tidy Data?

 Tidy data is a structured format where:

     - Each row represents one observation (e.g., a country in a given year).
     - Each column is a variable (e.g., GDP, life expectancy).
     - Each table represents a dataset (e.g., economic statistics).

 💡 Why use tidy data?
 
     - Easier to analyze: Works well with `groupby()`, `agg()`, and visualization libraries like Seaborn.
     - More readable: No redundant columns.
     - Plays nicely with Pandas and Seaborn.

In [ ]:
import pandas as pd

 ## Wide Format to Tidy (Long) Format

 In the dataset below, **each year's population** is in a **separate column**, which makes it a **wide format**.

 We can convert it to **tidy** format using `pd.melt()`.

In [ ]:
# population over time
df = pd.DataFrame({
    "country": ["USA", "Canada", "Brazil"],
    "1990": [253, 28, 149],
    "2000": [282, 31, 170],
    "2010": [309, 34, 192],
    "2020": [339, 38, 209],
    # "continent": ["North America", "North America", "South America"],
})

# Wide format
display(df)


 ### `pd.melt()`

 - `id_vars`: The columns that **stay the same** (identifiers).

 - `var_name`: Name of the **new column** that will hold the old column headers (years).

 - `value_name`: Name of the new column that will store the values (population in this case).

In [ ]:
df_tidy = df.melt(id_vars=["country"], var_name="year", value_name="population")
display(df_tidy)


 Notice how each row now represents **one country** in **one year**, and each column is **a single variable**.

 ## Converting Tidy (Long) Format Back to Wide Format

 - If you ever need to go back to **wide** format, you can use `pivot()` or `pivot_table()`.

In [ ]:
df_wide = df_tidy.pivot(index="country", columns="year", values="population")
display(df_wide)


 Here, each row is a **country**, and each column is a **year**—back to wide format.

 ## Summarizing Tidy Data with `groupby()`

 Tidy data makes it straightforward to **group** and **summarize**.



 ### `groupby("year")["population"].mean()`

 This computes the **mean population** for each year across **all countries**.

In [ ]:
for key,data in df_tidy.groupby("year"):
    display(key)
    display(data)

In [ ]:
df_year_mean = df_tidy.groupby("year")["population"].std()
display(df_year_mean)
# 

 ### Grouping by Multiple Columns

 We can also group by **both** `year` **and** `country`.

In [ ]:
df_year_country_sum = df_tidy.groupby(["year", "country"])["population"].sum()
display(df_year_country_sum)


 This returns a **multi-index Series**, showing the population **by year and by country**.

 ## `agg()` for Multiple Summaries

 The `agg()` function lets us apply **multiple aggregations** at once.

 For instance, we can find the **mean** and the **max** population per year.

In [ ]:
df_agg = df_tidy.groupby("year").agg({"population": ["mean", "max","sum"]})
display(df_agg)


 This shows the average (`mean`) population and the maximum (`max`) population in each **year**.

 ## Handling Missing Data

 Let's introduce some **missing values** to demonstrate `dropna()` and `fillna()`.

In [ ]:
# Create a copy with artificially introduced NaNs
df_missing = df_tidy.copy()
df_missing.loc[(df_missing["country"] == "Brazil") & (df_missing["year"] == "2020"), "population"] = None

display(df_missing)


 ### `dropna()`

 - **Removes** rows with missing values.

In [ ]:
df_dropped = df_missing.dropna(subset=["population"])
display(df_dropped)


 Brazil's 2020 row is **completely removed** because of the missing population.



 ### `fillna()`

 - **Fills** missing values with a specified value or method.

In [ ]:
df_filled = df_missing.fillna(0)
display(df_filled)


 Now, the missing value is replaced with `0`.

 ## Combining Data with `merge()`

 Often, you'll have **multiple DataFrames** that need to be joined.

 Below is an example for merging a **GDP** dataset with our **population** dataset.

In [ ]:
gdp_data = pd.DataFrame({
    "country": ["USA", "Canada", "Brazil"],
    "year": ["2020", "2020", "2020"],
    "gdp": [21439, 1736, 1445],  # GDP in billions (fictitious or approximate)
})

# Merging on both country and year
df_merged = df_tidy.merge(gdp_data, on=["country", "year"], how="left")
display(df_merged)


 We used `how="left"` so that **all rows from `df_tidy`** are preserved, even if some may not match in `gdp_data`.



 - `how="inner"` would only keep matching rows.

 - `how="outer"` keeps **all** rows from both DataFrames.

 ## Example: `sort_values()` and `query()`

 Tidy data also makes it easy to **sort** and **filter**.

In [ ]:
# Sort by population descending
df_sorted = df_tidy.sort_values("population", ascending=False)
display(df_sorted)


 ### `query()`

 An alternative way to filter rows:



 ```python

 df.query("population > 200 and country == 'USA'")

 ```



 is equivalent to



 ```python

 df[(df["population"] > 200) & (df["country"] == "USA")]

 ```

In [ ]:
df_filtered = df_tidy.query("population > 200 and country == 'USA'")
display(df_filtered)


 ## Tidy and Process the Billboard Dataset



 The Billboard dataset comes with **76 columns** corresponding to the chart position of each song from `x1st.week` through `x76th.week`. This is a classic example of **wide** data that needs to be **melted** (unpivoted) into a long (tidy) format.



 ### Goals

 1. **Load** the Billboard dataset from CSV.

 2. **Tidy** the data so each row represents one song in one week.

 3. **Calculate** the actual date for each week using `date.entered + week * 7 days`.

 4. **Split** the data into two tables:

    - A **songs** table with static song information.

    - A **positions** table with `(song_id, week, rank, date)`.

 5. **Save** the tidy data to **Feather** format in the same directory with `_tidy` suffix.

In [ ]:
import pandas as pd

# 1. Load the Billboard dataset
df_bill = pd.read_csv("../../Datasets/billboard.csv")

# Let's check a few columns to see the structure.
df_bill.head()

 The dataset has columns like:

 - **year**, **artist.inverted**, **track**, **time**, **genre** … (song info)

 - **date.entered**, **date.peaked** … (chart-related dates)

 - **x1st.week** through **x76th.week** … (chart positions over 76 weeks)



 We want to **melt** these weekly columns into a single `week` and `rank` column.

 Notice how each row is now **one song** in **one week**. However, the `week` column currently contains strings like `"x1st.week"`, `"x2nd.week"`, etc. Let's clean those up and create a numeric week column.

 Now, `week = 1, 2, 3, ... 76`. Next, we want to calculate the **exact date** on the chart for each row by adding `week * 7` days to `date.entered`.

 ### Split into Two Tables



 **Why split?** We often separate the **static** song info (e.g., artist, track, time, genre) from the **weekly** chart performance (week, rank, date).



 - **Songs Table**: Contains unique identifiers for each song plus basic metadata.

 - **Positions Table**: Contains `(song_id, week, rank, date)`, referencing the **song_id** from the songs table.

 Next, we merge this `song_id` back into our `df_tidy` so we can create the positions table.

 ### Create the Positions Table



 We only keep the **relevant columns** for weekly positions: `song_id`, `week`, `rank`, and `date`.

## 8.Playing with the data
 Now that we have our data in a tidy format, let's do some analysis.

### Only songs that reached top 10
We can use `query()` to filter the data for songs that reached the top 10 at least once. We will merge this back to the songs table to get the song details.

Now we need to remove duplicates to get a list of unique songs that reached the top 10.

### How long did each song stay in the top 10?

### In which week did each song reach the top 10?

 ### 9. Save Tidy Data to Feather



 We want to save:

 - The **tidy** DataFrame (`df_tidy`) to a single file with the suffix `_tidy`.

 - (Optionally) Also save **songs** and **positions** as separate Feather files if needed.